In [1]:
# Load the data
import pandas as pd
train_df = pd.read_csv("../../cleaned_dataset/split_data/model_selection/train.csv")
test_df = pd.read_csv("../../cleaned_dataset/split_data/model_selection/test.csv")

In [2]:
# Get all of the unique labels for train / test sets 
train_labels = list(train_df['room'].unique())
test_labels = list(test_df['room'].unique())

# Labels that appear in BOTH train and test
common_labels = list(set(train_labels) & set(test_labels))

# Now filter the train_df and test_df so that
# remove all record in train_df whose labels not in test labels 
# and remove all record in test_df whose labels not in train labels 
# therefore we can have a proper train and test df such that the labels appear 
# in train and test should be the same and vice versa
train_df = train_df[train_df['room'].isin(common_labels)].reset_index(drop=True)
test_df  = test_df[test_df['room'].isin(common_labels)].reset_index(drop=True)

In [3]:
# sanity check for label matching between train and test set
assert set(train_df['room'].unique()) == set(test_df['room'].unique())
print("Train/Test labels are now perfectly aligned")

Train/Test labels are now perfectly aligned


In [4]:
# check the distribution of classes in training set
train_df['room'].value_counts()

room
nurse station    488303
cafeteria        171492
kitchen          132819
hallway           63848
523               22169
511                8505
522                8079
cleaning           6232
506                4995
512                3773
520                3695
501                3415
517                1397
513                1152
502                 801
518                 252
Name: count, dtype: int64

In [5]:
# so now we could determine the minority classes would be room 502 and 518
minority_classes = ['502', '518']

# define the target number of records for these minority classes after we done 
# relabeling + concatenating with original training set
TARGET_NUM_502 = 1000 
TARGET_NUM_518 = 1200

In [6]:
# Function to add 25 beacon_1, beacon_2, ... , beacon_25
def add_beacon_features(df, num_beacons=25):
    df = df.copy()

    for i in range(1, num_beacons + 1):
        df[f'beacon_{i}'] = df['RSSI'].where(df['mac address'] == i, 0)

    return df

# Apply with our current train and test dataframe
train_df = add_beacon_features(train_df)
test_df = add_beacon_features(test_df)

In [7]:
train_df.head(1)

,timestamp,mac address,RSSI,room,beacon_1,beacon_2,beacon_3,beacon_4,beacon_5,beacon_6,...,beacon_16,beacon_17,beacon_18,beacon_19,beacon_20,beacon_21,beacon_22,beacon_23,beacon_24,beacon_25
0,2023-04-10 14:21:46+09:00,6,-93,kitchen,0,0,0,0,0,-93,...,0,0,0,0,0,0,0,0,0,0


In [8]:
test_df.head(1)

,timestamp,mac address,RSSI,room,beacon_1,beacon_2,beacon_3,beacon_4,beacon_5,beacon_6,...,beacon_16,beacon_17,beacon_18,beacon_19,beacon_20,beacon_21,beacon_22,beacon_23,beacon_24,beacon_25
0,2023-04-12 10:00:53+09:00,4,-92,cafeteria,0,0,0,-92,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Define the 6-beacon surrounding vectors for each room
# Format: [front-left, side-left, source, front, side-right, front-right]
# Based on the 5th floor map
# ONLY including rooms that appear in BOTH train and test sets

beacon_layout = {
    # Rooms from value_counts that are patient rooms
    '501': [16, 2, 1, 15, None, 13],      # Room 1
    '502': [17, 3, 2, 16, 1, 15],         # Room 2 - MINORITY CLASS
    '506': ["to be determined later"],    # Room 6
    '511': [23, 12, 11, 22, 10, 21],      # Room 11
    '512': [None, None, 12, 23, 11, 22],  # Room 12
    '513': [None, None, 13, None, 15, 1],      # Room 13
    '517': [2, 16, 17, 3, None, 5],      # Room 17
    '518': [None, 9, 18, 7, 20, 8],         # Room 18 - MINORITY CLASS
    '520': [7, 18, 20, 8, 21, 10],       # Room 20
    '522': [10, 21, 22, 11, 23, 12],      # Room 22
    '523': [11, 22, 23, 12, None, None],  # Room 23
}

# Classify rooms by matching type
full_matching_rooms = [room for room, beacons in beacon_layout.items() 
                       if None not in beacons]

partial_matching_rooms = list(beacon_layout.keys())

print(f"Total rooms defined: {len(beacon_layout)}")
print(f"Full matching rooms (complete 6 beacons): {full_matching_rooms}")
print(f"Total partial matching rooms: {len(partial_matching_rooms)}")
print(f"\nMinority classes: {minority_classes}")
print(f"Room 502 beacons: {beacon_layout['502']}")
print(f"Room 518 beacons: {beacon_layout['518']}")

Total rooms defined: 11
Full matching rooms (complete 6 beacons): ['502', '506', '511', '520', '522']
Total partial matching rooms: 11

Minority classes: ['502', '518']
Room 502 beacons: [17, 3, 2, 16, 1, 15]
Room 518 beacons: [None, 9, 18, 7, 20, 8]


In [12]:
# Test for BOTH minority classes: 502 and 518
print("=" * 80)
print("TESTING KL DIVERGENCE FOR MINORITY CLASSES")
print("=" * 80)

for minority_room in ['502', '518']:
    print(f"\n{'='*80}")
    print(f"MINORITY CLASS: Room {minority_room}")
    print(f"Sample count: {len(train_df[train_df['room'] == minority_room])}")
    print(f"Beacon vector: {beacon_layout[minority_room]}")
    print(f"{'='*80}\n")
    
    minority_vec = beacon_layout[minority_room]
    
    # Separate candidates into full and partial matching
    full_candidates = []
    partial_candidates = []
    
    for candidate_room, candidate_vec in beacon_layout.items():
        if candidate_room == minority_room:
            continue
        
        # Skip rooms with incomplete beacon layout definition
        if len(candidate_vec) != 6:
            print(f"Skipping Room {candidate_room}: beacon layout not fully defined")
            continue
        
        # Check if this would be full or partial matching
        valid_count = sum(1 for i in range(6) 
                         if minority_vec[i] is not None and candidate_vec[i] is not None)
        
        if valid_count == 6:
            full_candidates.append(candidate_room)
        else:
            partial_candidates.append(candidate_room)
    
    # Test FULL MATCHING candidates
    print("FULL MATCHING CANDIDATES:")
    print("-" * 80)
    full_results = {}
    for candidate_room in full_candidates:
        candidate_vec = beacon_layout[candidate_room]
        kl_score, match_type = calculate_kl_divergence(
            train_df, 
            minority_vec, 
            candidate_vec,
            minority_room,
            candidate_room
        )
        full_results[candidate_room] = kl_score
        print(f"Room {minority_room} vs Room {candidate_room}: KL = {kl_score:.6f} ({match_type})")
    
    if full_results:
        best_full = min(full_results, key=full_results.get)
        print(f"\n>>> BEST FULL MATCH: Room {best_full} with KL = {full_results[best_full]:.6f}")
    else:
        print("No full matching candidates available")
    
    # Test PARTIAL MATCHING candidates
    print(f"\n{'='*80}")
    print("PARTIAL MATCHING CANDIDATES:")
    print("-" * 80)
    partial_results = {}
    for candidate_room in partial_candidates:
        candidate_vec = beacon_layout[candidate_room]
        kl_score, match_type = calculate_kl_divergence(
            train_df, 
            minority_vec, 
            candidate_vec,
            minority_room,
            candidate_room
        )
        partial_results[candidate_room] = kl_score
        valid_beacons = sum(1 for i in range(6) 
                           if minority_vec[i] is not None and candidate_vec[i] is not None)
        print(f"Room {minority_room} vs Room {candidate_room}: KL = {kl_score:.6f} ({match_type}, {valid_beacons}/6 beacons)")
    
    if partial_results:
        best_partial = min(partial_results, key=partial_results.get)
        print(f"\n>>> BEST PARTIAL MATCH: Room {best_partial} with KL = {partial_results[best_partial]:.6f}")
    else:
        print("No partial matching candidates available")
    
    print("\n")

TESTING KL DIVERGENCE FOR MINORITY CLASSES

MINORITY CLASS: Room 502
Sample count: 801
Beacon vector: [17, 3, 2, 16, 1, 15]

Skipping Room 506: beacon layout not fully defined
FULL MATCHING CANDIDATES:
--------------------------------------------------------------------------------
Room 502 vs Room 511: KL = 22.934792 (full)
Room 502 vs Room 520: KL = 7.630353 (full)
Room 502 vs Room 522: KL = 2.949143 (full)

>>> BEST FULL MATCH: Room 522 with KL = 2.949143

PARTIAL MATCHING CANDIDATES:
--------------------------------------------------------------------------------
Room 502 vs Room 501: KL = 17.570795 (partial, 5/6 beacons)
Room 502 vs Room 512: KL = 21.277543 (partial, 4/6 beacons)
Room 502 vs Room 513: KL = 5.105597 (partial, 3/6 beacons)
Room 502 vs Room 517: KL = 15.095895 (partial, 5/6 beacons)
Room 502 vs Room 518: KL = 52.682750 (partial, 5/6 beacons)
Room 502 vs Room 523: KL = 2.610885 (partial, 4/6 beacons)

>>> BEST PARTIAL MATCH: Room 523 with KL = 2.610885



MINORITY CLA

In [13]:
def relabel_and_augment(df, minority_room, matched_room, target_count, beacon_layout):
    """
    Sample records from matched majority class, relabel them, and add to training data.
    
    Args:
        df: Original training DataFrame
        minority_room: Minority class room label (e.g., '502')
        matched_room: Best matched majority room label (e.g., '522')
        target_count: Target number of samples for minority class after augmentation
        beacon_layout: Dictionary mapping rooms to beacon vectors
    
    Returns:
        augmented_df: DataFrame with relabeled records added
        n_relabeled: Number of records that were relabeled
    """
    
    # Get current count of minority class
    current_count = len(df[df['room'] == minority_room])
    
    # Calculate how many records we need to relabel
    n_relabel = target_count - current_count
    
    if n_relabel <= 0:
        print(f"Room {minority_room} already has {current_count} samples (target: {target_count}). No relabeling needed.")
        return df, 0
    
    print(f"Room {minority_room}: Current = {current_count}, Target = {target_count}")
    print(f"Need to relabel {n_relabel} records from Room {matched_room}")
    
    # Get all records from matched majority room
    matched_room_data = df[df['room'] == matched_room].copy()
    
    if len(matched_room_data) < n_relabel:
        print(f"WARNING: Room {matched_room} only has {len(matched_room_data)} records, less than needed {n_relabel}")
        print(f"Will use all available records from Room {matched_room}")
        n_relabel = len(matched_room_data)
    
    # Randomly sample n_relabel records from matched room
    relabeled_data = matched_room_data.sample(n=n_relabel, random_state=42).copy()
    
    # RELABEL: Change room label from matched_room to minority_room
    # Keep ALL beacon RSSI values unchanged
    relabeled_data['room'] = minority_room
    
    print(f"Relabeled {len(relabeled_data)} records: {matched_room} → {minority_room}")
    
    # Concatenate with original training data
    augmented_df = pd.concat([df, relabeled_data], ignore_index=True)
    
    # Verify
    new_count = len(augmented_df[augmented_df['room'] == minority_room])
    print(f"After augmentation: Room {minority_room} has {new_count} samples")
    
    return augmented_df, n_relabel


# Apply relabeling for both minority classes
print("=" * 80)
print("RELABELING AND AUGMENTATION")
print("=" * 80)

# Store the matches we found
best_matches = {
    '502': '522',  # Room 502 → Room 522 (full matching)
    '518': '513',  # Room 518 → Room 513 (partial matching)
}

target_counts = {
    '502': TARGET_NUM_502,
    '518': TARGET_NUM_518
}

# Start with original training data
augmented_train_df = train_df.copy()

# Apply relabeling for each minority class
for minority_room in ['502', '518']:
    print(f"\n{'='*80}")
    matched_room = best_matches[minority_room]
    target_count = target_counts[minority_room]
    
    augmented_train_df, n_relabeled = relabel_and_augment(
        augmented_train_df,
        minority_room,
        matched_room,
        target_count,
        beacon_layout
    )
    print()

# Final verification
print("=" * 80)
print("FINAL CLASS DISTRIBUTION AFTER AUGMENTATION")
print("=" * 80)
print(augmented_train_df['room'].value_counts())
print(f"\nOriginal training size: {len(train_df)}")
print(f"Augmented training size: {len(augmented_train_df)}")
print(f"Added records: {len(augmented_train_df) - len(train_df)}")

RELABELING AND AUGMENTATION

Room 502: Current = 801, Target = 1000
Need to relabel 199 records from Room 522
Relabeled 199 records: 522 → 502
After augmentation: Room 502 has 1000 samples


Room 518: Current = 252, Target = 1200
Need to relabel 948 records from Room 513
Relabeled 948 records: 513 → 518
After augmentation: Room 518 has 1200 samples

FINAL CLASS DISTRIBUTION AFTER AUGMENTATION
room
nurse station    488303
cafeteria        171492
kitchen          132819
hallway           63848
523               22169
511                8505
522                8079
cleaning           6232
506                4995
512                3773
520                3695
501                3415
517                1397
518                1200
513                1152
502                1000
Name: count, dtype: int64

Original training size: 920927
Augmented training size: 922074
Added records: 1147
